# 4. 네이버 OPEN API

네이버 데이터 OPEN API를 사용하여 영화별 별점과 검색량 자료를 CSV파일로 저장

## 1) 네이버 별점

## 2) 네이버 datalab


In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import pandas as pd
from tqdm import tqdm
from tqdm import tqdm_notebook
from datetime import date
import glob
import multiprocessing
import os

In [2]:
# requests 모듈로 url 내용을 가져오고 BeautifulSoup로 내용 가공하기
start_date = date(2017,1,1)
end_date = date(2018,2,28)
# Dt=20170101
# m_name=[]
# movie_dict=[]
# movie_dict2=[]
# dff2 = pd.DataFrame()
#title_n = soup.find_all('div', 'tit3')

# for single_date in tqdm_notebook(pd.date_range(start_date, end_date)):
def naver_movie_daily_boxoffice(single_date):
    j=0
    movie_name=[]
    movie_point=[]
    targetDt = str(single_date)[0:4]+str(single_date)[5:7]+str(single_date)[8:10]
    url = "http://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=cnt&date="+targetDt
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "html.parser")
    b = soup.find_all('div','tit3')
    title_n = soup.find_all('div', 'tit3')
    # b = soup.find_all('div', 'tit3')[n]
    # 전체 영화의 제목 뽑아옴
    
    
    for n in range(0, len(title_n)):
        movie_name.append(b[n].find('a').text)

    # 전체 순위의 평점 뽑아옴
        movie_point.append(b[n].find('a')['href'][28:])
        
    dff = list(zip(movie_name,movie_point))
    dff = pd.DataFrame(dff)
#     dff = dff.drop_duplicates()
#     dff = dff.reset_index(drop=True)
    dff.to_csv('/home/ubuntu/movie/tmp_data/'+targetDt+'.csv',index=False)
    
    # 하루만 저장됨 전체가 저장되는 코드 필요함

In [3]:
cpu = os.cpu_count()
date_list = pd.date_range(start_date, end_date)
pool = multiprocessing.Pool(processes=cpu)
pool.map(naver_movie_daily_boxoffice,date_list)
pool.close()
pool.join()

In [4]:
tmp = glob.glob('/home/ubuntu/movie/tmp_data/*.csv')
dff = pd.DataFrame()
for i in tmp:
    df = pd.read_csv(i)
    dff = pd.concat([df,dff])
dff.to_csv('/home/ubuntu/movie/data/naver_boxoffice.csv',index=False)

In [5]:
tmp_data = glob.glob('/home/ubuntu/movie/tmp_data/*.csv')
for data in tmp_data:
    os.remove(data)

In [6]:
dff = pd.read_csv('/home/ubuntu/movie/data/naver_boxoffice.csv')
dff = dff.drop_duplicates()
dff2 = pd.DataFrame(columns=['2','3','4'])
dff = pd.concat([dff,dff2])
dff.to_csv('/home/ubuntu/movie/data/naver_boxoffice.csv',index=False)

In [7]:
# dff
import numpy as np
star1=[]
star2=[]
star3=[]
list1=[]
list2=[]


num1=df['1']
# for i in tqdm_notebook(range(0,len(df['1']))):
def naver_movie_rating(code):
    df = pd.read_csv('/home/ubuntu/movie/data/naver_boxoffice.csv')
    idx = np.where(df['1']==code)
    
    url1 = "https://movie.naver.com/movie/bi/mi/basic.nhn?code="
    page1 = urlopen(url1+str(code)[:-2])
    soup1 = BeautifulSoup(page1, "html.parser")
    b = soup1.find_all('span','st_on')
#     list1.append(df['1'][i])
#     list2.append(df['0'][i])
    print(len(b))
    if(len(b)<3):
        df['2'][idx[0][0]] = 0
        df['3'][idx[0][0]] = 0
        df['4'][idx[0][0]] = 0
    else:
        if(b[2]['style'][6:-1] == '0'):
            df['2'][idx[0][0]] = round(float(b[0]['style'][6:-1])/10,2)
            df['3'][idx[0][0]] = 0
            df['4'][idx[0][0]] = round(float(b[1]['style'][6:-1])/10,2)
        else:
            df['2'][idx[0][0]] = round(float(b[0]['style'][6:-1])/10,2)
            df['3'][idx[0][0]] = round(float(b[1]['style'][6:-1])/10,2)
            df['4'][idx[0][0]] = round(float(b[2]['style'][6:-1])/10,2)
    print(df['2'][idx[0][0]])
    print(df['3'][idx[0][0]])
    print(df['4'][idx[0][0]])
    df.to_csv('/home/ubuntu/movie/data/naver_boxoffice.csv',index=False)
# movie_dict2.append(list(zip(list1,list2,star1,star2,star3)))

In [ ]:
df = pd.read_csv('/home/ubuntu/movie/data/naver_boxoffice.csv')
code = 146506.0
idx = np.where(df['1']==code)
naver_movie_rating(code)


In [ ]:
import numpy as np

df = pd.read_csv('/home/ubuntu/movie/data/naver_boxoffice.csv')
url1 = "https://movie.naver.com/movie/bi/mi/basic.nhn?code="
page1 = urlopen(url1+str(df['1'][0])[:-2])
soup1 = BeautifulSoup(page1, "html.parser")
b = soup1.find_all('span','st_on')
len(b)

In [ ]:
from tqdm import tqdm
cpu = os.cpu_count()
df = pd.read_csv('/home/ubuntu/movie/data/naver_boxoffice.csv')
pool = multiprocessing.Pool(processes=cpu)
pool.map(naver_movie_rating,tqdm(df['1']))
pool.close()
pool.join()

In [ ]:
import pandas as pd
df = pd.read_csv('/home/ubuntu/movie/data/naver_boxoffice.csv')
# for i in tqdm_notebook(range(1,len(movie_dict2))):
#     df = df.append(movie_dict2[i], ignore_index=True)
df2 = df.drop_duplicates()
df2 = df2.reset_index(drop=True)
df2 = df2.rename(columns={df2.columns[0]:'movieNm',df2.columns[1]:'naver_movieCd', df2.columns[2]:'ntz_see_score', df2.columns[3]:'spc_score', df2.columns[4]:'ntz_score'})
df2.to_csv('/home/ubuntu/movie/data/naver_movie.csv',index=False)